<a href="https://colab.research.google.com/github/HemantTiwariGitHub/IntelligenceCodes/blob/master/FeatureExtrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 try:
    %tensorflow_version 2.x
except:
    pass

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.1.0-rc1
• Using TensorFlow Hub Version:  0.7.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
• GPU Device Not Found. Running on CPU


In [0]:
MOBILENET_V2_FEATURE_EXTRACTOR_URI = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
PIXEL =224
INPUT_IMAGE_SIZE = (PIXEL, PIXEL)
OUTPUT_FEATURE_VECTOR_SIZE = 1280

In [0]:
feature_extractor = hub.KerasLayer(MOBILENET_V2_FEATURE_EXTRACTOR_URI,
                                   input_shape=INPUT_IMAGE_SIZE + (3,), 
                                   output_shape=[OUTPUT_FEATURE_VECTOR_SIZE],
                                   trainable=False)

In [0]:
#Only Feature Extractor Model , No Blah :)
model = tf.keras.Sequential([
        feature_extractor
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
SAVED_MODEL_NAME = "image_mobilenetv2_featureExtractor"
tf.saved_model.save(model, SAVED_MODEL_NAME)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: image_mobilenetv2_featureExtractor/assets


INFO:tensorflow:Assets written to: image_mobilenetv2_featureExtractor/assets


In [0]:
loaded_model = tf.saved_model.load(SAVED_MODEL_NAME)

In [0]:
import numpy as np
import PIL.Image as Image

In [0]:
def get_uri_path(base_path, image_name):
    uri = pathlib.Path(base_path+'/' + image_name).as_uri()
    print("uri = " + uri)
    return  uri

In [0]:
def load_image(file_uri, image_name):
    print ("loading:"+file_uri)
    image = tf.keras.utils.get_file(image_name,file_uri)
    print("loaded:" +image) 
    return  image

In [0]:
def resize_image(image):
    print("resizing:" +image) 
    resized_img = Image.open(image).resize(INPUT_IMAGE_SIZE)
    #resized_img.show()
    resized_img_arr = np.array(resized_img)/255.0
    return resized_img_arr

In [0]:
def get_image_input(path, img_name):
    uri = get_uri_path(path, img_name)
    image = load_image(uri, img_name)
    resized = resize_image(image)
    return resized
    

In [0]:
def compute_cosine_distances(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim
    # results shape is n_a * n_b

    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b=True)
    return distance

In [0]:
cat1_fvec = model.predict(get_image_input(base_path, 'Cat1.jpg')[np.newaxis, ...])
cat2_fvec = model.predict(get_image_input(base_path, 'Cat2.jpg')[np.newaxis, ...])
ball1_fvec = model.predict(get_image_input(base_path, 'Ball1.jpg')[np.newaxis, ...])
ball2_fvec = model.predict(get_image_input(base_path, 'Ball2.jpg')[np.newaxis, ...])

uri = file:///content/Cat1.jpg
loading:file:///content/Cat1.jpg
loaded:/root/.keras/datasets/Cat1.jpg
resizing:/root/.keras/datasets/Cat1.jpg
uri = file:///content/Cat2.jpg
loading:file:///content/Cat2.jpg
loaded:/root/.keras/datasets/Cat2.jpg
resizing:/root/.keras/datasets/Cat2.jpg
uri = file:///content/Ball1.jpg
loading:file:///content/Ball1.jpg
loaded:/root/.keras/datasets/Ball1.jpg
resizing:/root/.keras/datasets/Ball1.jpg
uri = file:///content/Ball2.jpg
loading:file:///content/Ball2.jpg
loaded:/root/.keras/datasets/Ball2.jpg
resizing:/root/.keras/datasets/Ball2.jpg


In [1]:
val = compute_cosine_distances(cat1_fvec,cat2_fvec)
print (val)
val = compute_cosine_distances(ball1_fvec,ball2_fvec)
print (val)
val = compute_cosine_distances(cat1_fvec,ball1_fvec)
print (val)
val = compute_cosine_distances(cat1_fvec,ball2_fvec)
print (val)
val = compute_cosine_distances(cat2_fvec,ball1_fvec)
print (val)
val = compute_cosine_distances(cat2_fvec,ball2_fvec)
print (val)

NameError: ignored

In [3]:
%pip install visual_genome

In [0]:
from visual_genome import api as vg


In [0]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image as PIL_Image
import requests
from io import StringIO